In [1]:
__PRODUCTION__ = 1
__NAME__       = 'efficiency-grpah-kernel'
__WIDTH__      = 5.5  # NeurIPS 2021 text box width
__HEIGHT__     = 1.8

if __PRODUCTION__:
    from mplmagic2 import pgf
else:
    from mplmagic2 import svg

from mplmagic2 import SuperFigure, tex
import matplotlib.pyplot as plt
from matplotlib import patches
import matplotlib.patheffects as path_effects

print('This is how much space the figure will take up on letter paper')
SuperFigure.size_hint(__WIDTH__, __HEIGHT__, margin_left=0.5 * (8.5 - __WIDTH__));

This is how much space the figure will take up on letter paper


In [2]:
import functools
import numpy as np
import colorsys
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F
from torch import optim
import networkx as nx
import json
import pickle
import matplotlib

from symfac.experimental import RBFExpansionV2

In [3]:
d_tensor = torch.load('data/paper/d1')
target = torch.tensor(d_tensor, dtype=torch.float)

<ipython-input-3-0a4f1db08579>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(d_tensor, dtype=torch.float)


In [4]:
gd_10_losses, gd_10_times  = pickle.loads(open('gd_10.pickle', 'rb').read())
sgd_long_losses, sgd_long_times  = pickle.loads(open('sgd_long.pickle', 'rb').read())
sgd_10_losses, sgd_10_times  = pickle.loads(open('sgd_10.pickle', 'rb').read())
sgd_full  = pickle.loads(open('sgd_full.pickle', 'rb').read())

In [5]:
sgd_full_13  = pickle.loads(open('sgd_full_13.pickle', 'rb').read())
sgd_full_14  = pickle.loads(open('sgd_full_14.pickle', 'rb').read())
sgd_full_15  = pickle.loads(open('sgd_full_15.pickle', 'rb').read())
sgd_full_16  = pickle.loads(open('sgd_full_16.pickle', 'rb').read())

In [6]:
sgd_full_16

[0.0061504,
 0.0041449,
 0.0023808,
 0.0012769,
 0.0005753,
 0.0002058,
 7.15e-05,
 4.47e-05]

In [7]:
sgd_average = [1/5*(sgd_full[t] + sgd_full_13[t] + sgd_full_14[t] + sgd_full_15[t]+ sgd_full_16[t]) for t in range(len(sgd_full))]

In [8]:
#gk_losses = pickle.loads(open('data/paper/gk_losses_2.pickle', 'rb').read())
#gkl_losses = pickle.loads(open('data/paper/gk_long_losses.pickle', 'rb').read())


In [9]:
def find_svd(X,s):
    U, S, V = np.linalg.svd(X.detach().numpy())
    return round(float(F.mse_loss(torch.tensor((U[:, :s] * S[None, :s]) @ V[:s, :]), X)), 7)

In [10]:
s_vals = []
for s in range(1, target.shape[0]):
    s_vals.append(find_svd(target, s))

In [11]:
svd_10_losses = s_vals[:10]

In [12]:
checkpoints = [1,2,4,8,16,32,64,128]
checkpoints = [t - 1 for t in checkpoints]
checkpoints

[0, 1, 3, 7, 15, 31, 63, 127]

In [13]:
checkpoints

[0, 1, 3, 7, 15, 31, 63, 127]

In [14]:
svd_full = [s_vals[t] for t in checkpoints]

In [15]:
svd_10_losses

[0.0115783,
 0.0070272,
 0.0042515,
 0.0036036,
 0.0030903,
 0.0027092,
 0.0024542,
 0.0022272,
 0.0020658,
 0.0019309]

In [16]:
svd_full

[0.0115783,
 0.0070272,
 0.0036036,
 0.0022272,
 0.0014486,
 0.0009935,
 0.0006326,
 0.0003174]

In [17]:
fig = SuperFigure(figsize=(4, 4))
ax = fig.make_axes(0, 1, 0, 1)
ax.plot(np.arange(10) + 1, svd_10_losses)
ax.plot(np.array(checkpoints)[:4] + 1, svd_full[:4])
ax.set_xscale('log')
ax.set_yscale('log')
plt.show()

In [22]:
fig = SuperFigure(plt.figure(figsize=(__WIDTH__, __HEIGHT__), dpi=300))
ax_canvas = fig.make_axes(
    left=0, right=1, top=0, bottom=1, zorder=-100,
    style='blank' if __PRODUCTION__ else None
)
ax_canvas.set_xlim([0, 1])
ax_canvas.set_ylim([0, 1])

# draw the axes grid
x0 = 0.01
dx = 0.255
dx2 = 0.01
w = 0.205
axs = [
    fig.make_axes(
        #left=x0 + dx * i - dx2*(i//3),
        left=x0 + dx * i + (dx2 if i >= 2 else 0),
        width=w,
        top=0.165,
        width_to_height=1.0,
        style='modern'
    ) for i in range(4)
]

def draw_bars(ax, h, offset, **kwargs):
    ax.bar(np.arange(len(h)) + offset, h, **kwargs)

image_style = dict(
    vmin=0,
    vmax=1,
    cmap='Spectral'
)
plot_style = dict(
    lw=0.75,
    marker='o',
    markersize=2.5
)
svd_style = dict(
    color='#404040',
)
rbf_style = dict(
    color='#2080FF',
)
srbf_style = dict(
    color='#FF4060',
)


axs[0].imshow(target, **image_style)

axs[1].plot(checkpoints, sgd_average, **plot_style, **srbf_style)
axs[1].plot(checkpoints, svd_full, **plot_style, **svd_style)

axs[2].plot(range(1, 11), svd_10_losses, **plot_style, **svd_style)
axs[2].plot(range(1, 11), gd_10_losses, **plot_style, **rbf_style)
axs[2].plot(range(1, 11), sgd_10_losses, **plot_style, **srbf_style)

bar_width = 0.25
bar_style = dict(
    width = bar_width
)
# draw_bars(axs[3], [gd_10_times[t]/sgd_10_times[t] for t in range(10)], -0.5 * bar_width, **bar_style, **srbf_style)
draw_bars(axs[3], [gd_10_times[t] for t in range(10)], -0.5 * bar_width, **bar_style, **rbf_style)
draw_bars(axs[3], [sgd_10_times[t] for t in range(10)], 0.5 * bar_width, **bar_style, **srbf_style)
# draw_bars(axs[3], data['times']['rbf'], 0.5 * bar_width, **bar_style, **rbf_style)


title_style = dict(
    fontsize=8,
    y=0.945,
    va='bottom'
)

tick_style = dict(
    fontsize=7,
)
xlabel_style = dict(
    fontsize=8,
    labelpad=1.0,
)
ylabel_style = dict(
    fontsize=8,
    labelpad=0.75,
)

ticks = [0,250]
axs[0].set_xbound(lower=0, upper=250)
axs[0].set_ybound(lower=0, upper=250)
axs[0].set_xticks(ticks)
axs[0].set_yticks([])
axs[0].set_xticklabels(['%d' % x for x in [0, 250]], **tick_style)
axs[0].set_xlabel('Molecule ID', **xlabel_style)

for ax in axs[1:2]:
    ax.plot(
        [1.05, 8, 8, 1.05, 1.05],
        [1e-3, 1e-3, 0.98e-2, 0.98e-2, 1e-3],
        color='k',
        lw=0.5,
        ls=(2, (1, 3)),
        dash_capstyle='round'
    )
    polygon_style = dict(
        edgecolor='none',
        clip_on=False,
    )
    ax.add_patch(
        patches.Rectangle(
            (1.03, 1e-3),
            8 - 1.03, 1e-2 - 1e-3,
            facecolor='#E0E0E0',
            **polygon_style
        )
    )
    ax.add_patch(
        patches.FancyArrow(
            8, 1.5 * 1e-5**0.5, 512, 0,
            width=0.00025,
            head_width=0.0015,
            head_length=80,
            facecolor='#404040',
            length_includes_head=True,
            **polygon_style
        )
    )

    # extra_ticks = np.linspace(-1, -3, 3)
#     for j, ref in enumerate([1e-2, 1e-3]):
#         r_rbf, r_svd = (
#             np.flatnonzero(np.array(sgd_average) <= ref)[0] + 1,
#             np.flatnonzero(np.array(svd_full) <= ref)[0] + 1
#         )
#         ax.axhline(
#             ref,
#             color='w',
#             lw=0.5,
#             ls=(2, (1.7, 3.1)),
#             dash_capstyle='round',
#             zorder=30
#         )
#         ax.text(
#             128, ref * 0.8, f'RBF {r_rbf} / {r_svd} SVD  ',
#             ha='right',
#             va='top',
#             fontsize=7,
#             zorder=20,
#             path_effects=[
#                 path_effects.Stroke(linewidth=1.5, foreground='w'),
#                 path_effects.Normal()
#             ]        
#         )
#         patch_w = 32
#         ax.add_patch(
#             patches.Rectangle(
#                 (128 - patch_w, ref * 0.25), patch_w, 0.75 * ref,
#                 color='w',
#                 alpha=0.8,
#                 zorder=10
#             ),
#         )
            
    ax.set_xscale('log', base=2)
    ax.set_yscale('log', base=10)
    ax.minorticks_off()
    xticks = np.arange(8)
    yticks = np.linspace(-5, -2, 4)
    ax.set_xbound(lower=2**xticks.min(), upper=2**xticks.max())
    ax.set_ybound(lower=10**yticks.min(), upper=10**yticks.max())
    ax.set_xticks(2**xticks)
    ax.set_yticks(10**yticks)
    ax.set_xticklabels([fr'{2**t:.0f}' for t in xticks], **tick_style)
    ax.set_yticklabels([fr'${tex.pow(10, int(t))}$' for t in yticks], **tick_style)
    ax.set_xlabel('Components', **xlabel_style)
    ax.set_ylabel('MSE Loss', **ylabel_style)

for ax in axs[2:3]:
    ax.set_yscale('log', base=10)
    ax.minorticks_off()
    xticks = np.arange(1, 11)
    yticks = np.linspace(-3, -2, 2) 
    ax.set_xbound(lower=xticks.min(), upper=xticks.max())
    ax.set_ybound(lower=0.5 * 10**yticks.min(), upper=10**yticks.max())
    ax.set_xticks(xticks)
    ax.set_yticks(10**yticks)
    ax.set_xticklabels(['%d' % x for x in xticks], **tick_style)
    ax.set_yticklabels([fr'${tex.pow(10, int(t))}$' for t in yticks], **tick_style)
    ax.set_xlabel('Components', **xlabel_style)
    ax.set_ylabel('MSE Loss', **ylabel_style)
    
for ax in axs[3:4]:
    xticks = np.arange(1, 9)
    # yticks = [1,2,3,4]
    yticks = np.linspace(0, 50, 6)
    ax.set_xbound(lower=xticks.min() - 0.5, upper=xticks.max() + 0.5)
    ax.set_ybound(lower=yticks.min(), upper=yticks.max())

    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    ax.set_xticklabels(['%d' % x for x in xticks], **tick_style)
    ax.set_yticklabels(['%d' % y for y in yticks], **tick_style)
    ax.set_xlabel('Components', **xlabel_style)
    ax.set_ylabel('Time (s)', **ylabel_style)

for ax in axs[1:]:
    ax.tick_params(
        axis='y', which='both', length=1, direction='in', pad=1
    )

axs[0].set_title(r'\textbf{Molecular Distance}', **title_style)
axs[1].set_title(r'\textbf{Accuracy}', **title_style)
axs[2].set_title(r'Accuracy', **title_style)
axs[3].set_title(r'Time to Accuracy', **title_style)

ax_zoomin = fig.make_axes(
    left=axs[2].left,
    right=axs[3].right,
    top=0,
    bottom=1,
    style='blank'
)
ax_zoomin.set_xlim([0, 1])
ax_zoomin.set_ylim([0, 1])
overtitle_style = dict(
    fontsize=8,
    va='top',
    ha='center'
)
ax_zoomin.text(
    0.5, 0.985,
    r'\textbf{Comparison of GD and SGD} (k$\leq$8)',
    **overtitle_style,
)
ax_zoomin.axhline(0.915, lw=0.5, color='k')

ax_canvas.bar([-1], [0], **bar_style, **svd_style, label='SVD')
ax_canvas.bar([-1], [0], **bar_style, **rbf_style, label='RBF by GD')
ax_canvas.bar([-1], [0], **bar_style, **srbf_style, label='RBF by SGD')
ax_canvas.legend(
    loc='upper center',
    bbox_to_anchor=(0.5, 0.09),
    ncol=3,
    frameon=False,
    fontsize=6
)

if __PRODUCTION__:
    fig.savefig(f'pgf/{__NAME__}.pgf', dpi=300)
else:
    fig.savefig(f'svg/{__NAME__}.svg', dpi=300)
plt.show()

In [23]:
!make -f Makefile.figures fig-"$__NAME__".pdf 2>&1 | tail -n 1

Successfully created fig-efficiency-grpah-kernel.pdf


END
---